In [1]:
import scipy.io
import numpy as np
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
from Encoding import load
dataset_name = "Pill_Mono"
par_data, bc = load(dataset_name,data_type="par_data"), load(dataset_name,data_type="bc")

In [ ]:
bc[0]

In [ ]:
np.max(par_data[0][0],axis=0)

In [ ]:
from Encoding import ProjectPointsToCylinder, ProjectPointsToHyperplane
t = 0
bc_step = bc[0].copy()
bc_step[0]+=bc_step[1]*(t+1)
par = par_data[0][t][:,:3]
def CheckCylindricalBC(points,cyl):
    point_to_cyl_origin = points-cyl[:3]
    cyl_axis = cyl[3:6]/np.linalg.norm(cyl[3:6])
    radius = cyl[6]

    axis_projection = point_to_cyl_origin*cyl_axis*cyl_axis
    axis_projection += cyl[:3]
    dist_axis = np.linalg.norm(points-axis_projection,axis=1)

    if cyl[-2] == 1: 
        bc_bool = dist_axis <= radius
    if cyl[-2] == -1:
        bc_bool = dist_axis >= radius

    return bc_bool 

def CheckPlanarBC(points,plane):
    wallpoints = ProjectPointsToHyperplane(points,plane)  
    vector = points-wallpoints 
    vector /= np.linalg.norm(vector,axis=1,keepdims=True) 
    normal_vector = plane[3:6]
    bc_bool = np.all(vector == normal_vector,axis=1)    
    return bc_bool

def ValidateBC(par,bc_step):
    bc_bool = np.empty([par.shape[0],bc_step.shape[1]])
    for wall_id,wall in enumerate(bc_step[0]):
        if wall[-1] == 1:
            bc_bool[:,wall_id] = CheckCylindricalBC(par,wall)
        if wall[-1] == 0:
            bc_bool[:,wall_id] = CheckPlanarBC(par,wall)
    return bc_bool

def ParticlesOutsideBoundary(data_list,bc_rollout):
    outside_particles = np.zeros(len(data_list))
    for t,data,bc_step in enumerate(zip(data_list,bc_rollout)):
        bc_bool = ValidateBC(data.pos[data.mask],bc_step)
        par_in_bounds = np.all(bc_bool,axis=1)
        num_par_out_bounds = np.sum(~par_in_bounds).item()
        outside_particles[t] = num_par_out_bounds
    return outside_particles


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
vector = axis_projection
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)
vector = par
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)

In [ ]:
vector = axis_projection
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(x,z,'o',color='b')
vector = par
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(x,z,'o',color='r')

In [ ]:
from Encoding import ProjectPointsToCylinder, ProjectPointsToHyperplane, ConstructTopology
par = par_data[0][0]
bc_t = bc[0]
super_topology = ConstructTopology(par,bc_t,4)

In [ ]:
from Encoding import TopologySlice, CheckForWallContact
from Evaluation import NumpyGroupby


from Encoding import TopologyFromPlausibleTopology
print(bc_t.shape)
topology = TopologyFromPlausibleTopology(super_topology,par,bc_t)

In [ ]:
topology.shape, super_topology.shape

In [ ]:

from Encoding import GetVirtualParticlesCoords
P_wall, normal_vectors = GetVirtualParticlesCoords(par,topology,bc_t)

P_virtual = np.concatenate((P_wall,
                            np.zeros((P_wall.shape[0],3)),                      
                            normal_vectors,
                            np.zeros((P_wall.shape[0],1))),                      
                            axis=1)
print(P_wall)
top_new = topology.copy()
n_new = P_virtual.shape[0]
n_par = par.shape[0]
top_new[-n_new:,1] = np.arange(n_par,n_new+n_par,1)
print(top_new)
    

In [ ]:
np.arange(n_new,n_new+n_par,1)

In [ ]:
bc.shape

In [ ]:
from Encoding import BCEncoding

P_virtual, top_new = BCEncoding(par_data[0][0],top,bc_t)
P_virtual, top_new

In [ ]:
point_on_wall = ProjectPointsToCylinder(particles,cyl)
#point_on_wall = ProjectPointsToHyperplane(particles,wall)


In [ ]:
from Plotting import Plot3DVectors
#fig = plt.figure()
#ax = fig.add_subplot(projection='3d')
#Plot3DVectors(ax,axis_projection,s)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
vector = proj+wall[:3]
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)